In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow.python.data import Dataset
import keras
from keras.utils import to_categorical
from keras import models
from keras import layers

from sklearn.model_selection import KFold

C:\Users\amanda\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load and preprocess data 

In [98]:
df_train = pd.read_csv('train.csv', index_col=0)
df_test = pd.read_csv('test.csv', index_col=0)

In [99]:
df_train = df_train.drop('opened_position_qty ', axis=1)
df_test = df_test.drop('opened_position_qty ', axis=1)
df_train = df_train.drop('closed_position_qty', axis=1) 
df_test = df_test.drop('closed_position_qty', axis=1)
#df_train = df_train.drop('transaction_qty', axis=1) 
#df_test = df_test.drop('transaction_qty', axis=1)
#df_train = df_train.drop('d_open_interest', axis=1) 
#df_test = df_test.drop('d_open_interest', axis=1)

In [45]:
# identify and replace missing values
print(df_train.isnull().sum())
print(df_test.isnull().sum())

print(df_train['opened_position_qty '].mean())
print(df_train['opened_position_qty '].median())

print(df_train['closed_position_qty'].mean())
print(df_train['closed_position_qty'].median())

# replace missing values with median (less sensitive to outliers)
df_train['opened_position_qty '].fillna(df_train['opened_position_qty '].median(),inplace=True)
df_test['opened_position_qty '].fillna(df_train['opened_position_qty '].median(),inplace=True)
df_train['closed_position_qty'].fillna(df_train['closed_position_qty'].median(),inplace=True)
df_test['closed_position_qty'].fillna(df_train['closed_position_qty'].median(), inplace=True)

last_price                   0
mid                          0
opened_position_qty     172460
closed_position_qty     172460
transacted_qty               0
d_open_interest              0
bid1                         0
bid2                         0
bid3                         0
bid4                         0
bid5                         0
ask1                         0
ask2                         0
ask3                         0
ask4                         0
ask5                         0
bid1vol                      0
bid2vol                      0
bid3vol                      0
bid4vol                      0
bid5vol                      0
ask1vol                      0
ask2vol                      0
ask3vol                      0
ask4vol                      0
ask5vol                      0
y                            0
dtype: int64
last_price                  0
mid                         0
opened_position_qty     53656
closed_position_qty     53656
transacted_qty              0


In [100]:
# Normalize data
from sklearn import preprocessing

x_train = df_train[df_train.columns[:24]]
y_train = df_train['y']
x_test = df_test

# Normalize training data by subtracting mean and scaling to unit variance
std_scale = preprocessing.StandardScaler().fit(x_train)
x_train_norm = std_scale.transform(x_train)
x_train = pd.DataFrame(x_train_norm, index=x_train.index, columns=x_train.columns)

# Normalize testing data by using mean and SD of training set
x_test_norm = std_scale.transform(x_test)
x_test = pd.DataFrame(x_test_norm, index=x_test.index, columns=x_test.columns) 

## Build and train model

In [101]:
model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(x_train.shape[1],)),
    keras.layers.Dense(50, activation=tf.nn.relu),
    keras.layers.Dropout(0.1, noise_shape=None, seed=None),
    keras.layers.Dense(60, activation=tf.nn.relu),
    keras.layers.Dropout(0.1, noise_shape=None, seed=None),
    keras.layers.Dense(100, activation=tf.nn.relu),
    keras.layers.Dropout(0.1, noise_shape=None, seed=None),
    keras.layers.Dense(100, activation=tf.nn.relu),
    keras.layers.Dropout(0.1, noise_shape=None, seed=None),
    keras.layers.Dense(100, activation=tf.nn.relu),
    keras.layers.Dropout(0.1, noise_shape=None, seed=None),
    keras.layers.Dense(70, activation=tf.nn.relu),
    keras.layers.Dropout(0.1, noise_shape=None, seed=None),
    keras.layers.Dense(50, activation=tf.nn.relu),
    keras.layers.Dropout(0.1, noise_shape=None, seed=None),
    keras.layers.Dense(30, activation=tf.nn.relu),
    keras.layers.Dropout(0.1, noise_shape=None, seed=None),
    keras.layers.Dense(20, activation=tf.nn.relu),
    keras.layers.Dropout(0.1, noise_shape=None, seed=None),
    keras.layers.Dense(2, activation=  'softmax')
])

In [102]:
model.compile(optimizer = keras.optimizers.adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [103]:
train_model = model.fit(x_train, y_train, epochs= 10, batch_size = 128, validation_split=0.2)

Train on 473904 samples, validate on 118476 samples
Epoch 1/10
473904/473904 [==============================] - 52s 110us/step - loss: 0.6288 - acc: 0.6492 - val_loss: 0.6448 - val_acc: 0.6212
Epoch 2/10
473904/473904 [==============================] - 51s 108us/step - loss: 0.6244 - acc: 0.6516 - val_loss: 0.6421 - val_acc: 0.6319
Epoch 3/10
473904/473904 [==============================] - 54s 113us/step - loss: 0.6234 - acc: 0.6522 - val_loss: 0.6427 - val_acc: 0.6213
Epoch 4/10
473904/473904 [==============================] - 54s 114us/step - loss: 0.6232 - acc: 0.6525 - val_loss: 0.6408 - val_acc: 0.6277
Epoch 5/10
473904/473904 [==============================] - 54s 114us/step - loss: 0.6229 - acc: 0.6528 - val_loss: 0.6435 - val_acc: 0.6212
Epoch 6/10
473904/473904 [==============================] - 53s 112us/step - loss: 0.6226 - acc: 0.6528 - val_loss: 0.6412 - val_acc: 0.6236 - loss: 0.6225 - ac
Epoch 7/10
473904/473904 [==============================] - 52s 111us/step - loss:

## Save results

In [104]:
df_test['Predicted'] = model.predict_proba(x_test)[:,1]
df_test[['Predicted']].to_csv('submission5.csv')